[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/vector-search-lab/)


In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AnyMessage
from langgraph.runtime import get_runtime
from dataclasses import dataclass
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel


class AgentResponse(BaseModel):
    # ensure strict compliance with the schema
    weather_conditions: str
    temperature: float
    address: str


@dataclass
class ContextSchema:
    user_name: str


# Load environment variables from .env file
load_dotenv()


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


def get_address(user: str) -> str:
    """Get users address"""
    return f"{user} lives at London, Elephant and Castle"


def get_products_in_store(store: str) -> str:
    """Get products for a given store"""
    return (
        f"The store {store} has the following products: {['apple', 'banana', 'cherry']}"
    )


def prompt(state: AgentState) -> list[AnyMessage]:
    runtime = get_runtime(ContextSchema)
    system_msg = (
        f"You are a helpful assistant. Address the user as {runtime.context.user_name}."
    )
    return [{"role": "system", "content": system_msg}] + state["messages"]


agent = create_react_agent(
    model=init_chat_model(model="openai:gpt-4o-mini"),
    # the AI model will be queried to choose which tool to use based on the user's request
    tools=[get_weather, get_address, get_products_in_store],
    prompt=prompt,
    context_schema=ContextSchema,
    checkpointer=InMemorySaver(),
    response_format=AgentResponse,
)


In [2]:
def callAgent(message: str, user_name: str, thread_id: str):
    result = agent.invoke(
        {"messages": [{"role": "user", "content": message}]},
        context=ContextSchema(user_name=user_name),
        config={"thread_id": thread_id},
    )
    return result["messages"][-1].content

In [3]:
callAgent("what is the weather in sf?", "Ricardo Villanueva", "123")

'The weather in San Francisco is always sunny!'

In [5]:
callAgent(
    "What products are in the store in London that are blue?",
    "Ricardo Villanueva",
    "123",
)

'The store in London does not have any blue products available. The available products are apple, banana, and cherry, none of which are blue.'